In [ ]:
#####################
'''LIBRARY-IMPORTS'''
#####################

#Importing all necessary libraries
import pandas as pd #used for data manipluation
import numpy as np #provides support for numerical operations

In [ ]:
#################################
'''DATA-LOADING & MANIPULATION'''
#################################

#read the data from the csv files and store it in a dataframe, also add a column for the year (of rating, for composite key and general overview)
df18 = pd.read_csv(r"C:\Users\D\OneDrive - Grunenthal Group\Desktop\VSC - Py\GRTend\Data\report_Erik_EC_Data_18201231.csv", delimiter= ",", encoding  = 'ISO-8859-1', low_memory = False)
df19 = pd.read_csv(r"C:\Users\D\OneDrive - Grunenthal Group\Desktop\VSC - Py\GRTend\Data\report_Erik_EC_Data_20181231.csv", delimiter= ",", encoding  = 'ISO-8859-1', low_memory = False)
df20 = pd.read_csv(r"C:\Users\D\OneDrive - Grunenthal Group\Desktop\VSC - Py\GRTend\Data\report_Erik_EC_Data_20181231.csv", delimiter= ",", encoding  = 'ISO-8859-1', low_memory = False)
df21 = pd.read_csv(r"C:\Users\D\OneDrive - Grunenthal Group\Desktop\VSC - Py\GRTend\Data\report_Erik_EC_Data_20181231.csv", delimiter= ",", encoding  = 'ISO-8859-1', low_memory = False)
df22 = pd.read_csv(r"C:\Users\D\OneDrive - Grunenthal Group\Desktop\VSC - Py\GRTend\Data\report_Erik_EC_Data_20181231.csv", delimiter= ",", encoding  = 'ISO-8859-1', low_memory = False)

df18['Year'] = 2018
df19['Year'] = 2019
df20['Year'] = 2020
df21['Year'] = 2021
df22['Year'] = 2022

In [ ]:
#Define common columns and pass only those columns to the dataframe
common_columns = [
    'User/Employee ID', 'Username', 'Direct Reports', 'In current position since',
    'Date Of Birth', 'Country Of Birth', '1. Citizenship', '2. Citizenship',
    'Gender', 'Employee Status', 'Position', 'Position since', 'Position Level',
    'Legal Entity', 'Country (LE)', 'Global Function', 'Employee Classification',
    'New Job since', 'FTE', 'Job Function', 'Job Title', 'Employment Details Hire Date',
    'Employment Details Original Start Date', 'Employment Details Calculation Start Date',
    'Employment Details Termination Date', 'Year'
]

df18 = df18[common_columns]
df19 = df19[common_columns]
df20 = df20[common_columns]
df21 = df21[common_columns]
df22 = df22[common_columns]

In [ ]:
#define date columns and transform them to datetime
date_columns = ['In current position since', 'Date Of Birth', 'Employment Details Hire Date',
                'Employment Details Original Start Date', 'Employment Details Calculation Start Date',
                'Employment Details Termination Date', 'Position since', 'New Job since']

for col in date_columns:
    df18[col] = pd.to_datetime(df18[col], dayfirst= True, errors= 'coerce')
    df19[col] = pd.to_datetime(df19[col], dayfirst= True, errors= 'coerce')
    df20[col] = pd.to_datetime(df20[col], dayfirst= True, errors= 'coerce')
    df21[col] = pd.to_datetime(df21[col], dayfirst= True, errors= 'coerce')
    df22[col] = pd.to_datetime(df22[col], dayfirst= True, errors= 'coerce')

In [ ]:
#define file paths for the output files
dataset_file_path = r"C:\Users\D\OneDrive - Grunenthal Group\Desktop\VSC - Py\GRTend\Data\data.csv" #for the final dataset
gender_file_path = r"C:\Users\D\OneDrive - Grunenthal Group\Desktop\VSC - Py\GRTend\Data\gender_data.csv" #for gender imputation/ model building

In [ ]:
#create a single dataframe for the employees, check DataFrame shape
employee_df = pd.concat([df18, df19, df20, df21, df22], ignore_index=True)

In [ ]:
#load csv and create dataframe for ratings, check DataFrame shape and create 'Year' for composite key and general overview
ratings_df = pd.read_csv(r'C:\Users\D\OneDrive - Grunenthal Group\Desktop\VSC - Py\GRTend\Data\_Ratings_ready to use.csv', delimiter= ';', encoding  = 'ISO-8859-1', low_memory = False)

ratings_df.rename(columns={'username': 'Username'}, inplace=True)

ratings_df['start-date'] = pd.to_datetime(ratings_df['start-date'], errors= 'coerce')

ratings_df['Year'] = ratings_df['start-date'].dt.year

In [ ]:
#create composite key in ratings_df and employee_df and merge to Dataset

employee_df['composite_key'] = employee_df['Username'] + '_' + employee_df['Year'].astype(str)

ratings_df['composite_key'] = ratings_df['Username'] + '_' + ratings_df['Year'].astype(str)

dataset = pd.merge(employee_df, ratings_df, on = 'composite_key', how = 'outer')

dataset = dataset.dropna(subset=['rating'])

print("Shape of merged 'employee' DataFrame: ", employee_df.shape)
print("Shape of 'Ratings' DataFrame: ", ratings_df.shape)
print("the shape of the 'Dataset' is: ", dataset.shape)

dataset.info()

In [ ]:
###############################
'''TESTS/ FUNCTIONS/ HELPERS'''
###############################

#check for outliers in numerical columns with the use of IQR (Interquartile Range/ Interquartilsabstand (deskripitve Statistik))
def calculate_outliers(dataset, column):
    Q1 = dataset[column].quantile(0.25)
    Q3 = dataset[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return dataset[(dataset[column] < lower_bound) | (dataset[column] > upper_bound)]


#define function to calculate modified z-score for outlier detection
def calculate_modified_z_score(dataset, column):
    median = dataset[column].median()
    median_absolute_deviation = np.median(np.abs(dataset[column] - median))
    modified_z_scores = 0.6745 * (dataset[column] - median) / median_absolute_deviation
    return dataset[np.abs(modified_z_scores) > 3.5]

def impute_legal_entity(dataset):
    country_mapping = {
        'DE': 'Germany', 'AT': 'Austria', 'CH': 'Switzerland', 'BE': 'Belgium', 'NL': 'Netherlands', 'LU': 'Luxembourg',
        'GB': 'United Kingdom', 'FR': 'France', 'IT': 'Italy', 'ES': 'Spain', 'PT': 'Portugal', 'GR': 'Greece',
        'PA': 'Panama', 'US': 'United States', 'CA': 'Canada', 'MX': 'Mexico', 'BR': 'Brazil', 'AR': 'Argentina',
        'CL': 'Chile', 'PE': 'Peru', 'CO': 'Colombia', 'VE': 'Venezuela', 'EC': 'Ecuador', 'UY': 'Uruguay',
        'NO': 'Norway', 'SE': 'Sweden', 'FI': 'Finland', 'DK': 'Denmark', 'PL': 'Poland', 'CZ': 'Czech Republic',
        'IE': 'Ireland', 'HU': 'Hungary', 'RO': 'Romania', 'RU': 'Russia', 'TR': 'Turkey',
    }

    missing_legal_entity = dataset['Legal Entity'].isnull()

    dataset.loc[missing_legal_entity, 'Legal Entity'] = dataset.loc[missing_legal_entity, 'Country (LE)'].apply(
        lambda x: f"GRT_{country_mapping.get(x, 'Unknown').upper()}"
    )
    return dataset

In [ ]:
#########################
'''FEATURE ENGINEERING''' #stands for Feature Creation, handling missing values, encoding, scaling, extracting info, feature selection, feature extraction (reducing dimensionality)
#########################

#impute ['Employee Status'] with 'Active' if the Employee isnt terminated, else with 'Passive - termination'
status_null_mask = dataset['Employee Status'].isnull()
termination_null_mask = dataset['Employment Details Termination Date'].isnull()
dataset.loc[status_null_mask & termination_null_mask, 'Employee Status'] = 'Active'
dataset.loc[status_null_mask & ~termination_null_mask, 'Employee Status'] = 'Passive - termination'

#impute missing values for ['Employee Classification'] with 'Employee' (most common value and statiscally likely)
dataset['Employee Classification'].fillna('Employee', inplace = True)

#impute ['Country (LE)'] and ['1. Citizenship'] - vice versa depending what we know
dataset['Country (LE)'] = dataset['Country (LE)'].fillna(dataset['1. Citizenship'])
dataset['1. Citizenship'] = dataset['1. Citizenship'].fillna(dataset['Country (LE)'])

#impute ['Country Of Birth'] with ['1. Citizenship'] if missing (most likely)
dataset['Country Of Birth'] = dataset['Country Of Birth'].fillna(dataset['1. Citizenship'])

#create ['Native'] column to check if the employee is native to the country of the legal entity
dataset['Native'] = np.where(dataset['1. Citizenship'] == dataset['Country (LE)'], True, False)

#impute ['Legal Entity'] with mapping of ['Country (LE)'] and ['Legal Entity']
dataset = impute_legal_entity(dataset)

#create ['Birth year'] column to extract the year of birth from the date of birth
dataset['Birth year'] = dataset['Date Of Birth'].dt.year

#create ['In current position since'] column to extract the year of the current position start date
dataset['In current position since'] = dataset['In current position since'].dt.year

#create ['New Job since'] column to extract the year of the new job start date
dataset['New Job since'] = dataset['New Job since'].dt.year

#create ['Employment Details Hire Date'] column to extract the year of the hire date
dataset['Employment Details Hire Year'] = dataset['Employment Details Hire Date'].dt.year

#create ['Employment Details Termination Date'] column to extract the year of the termination date
dataset['Employment Details Termination Year'] = dataset['Employment Details Termination Date'].dt.year

#create ['Employment Details Calculation Start Date'] column to extract the year of the calculation start date
dataset['Employment Details Calculation Start Year'] = dataset['Employment Details Calculation Start Date'].dt.year

#create ['Employment Details Original Start Year'] column to extract the year of the original start date
dataset['Employment Details Original Start Year'] = dataset['Employment Details Original Start Date'].dt.year

#create ['Position since'] column to extract the year of the position start date
dataset['Position since'] = dataset['Position since'].dt.year

#catch outliers in the ['Birth year'] column, impute wrong values with the median
birth_year_median = dataset.loc[dataset['Birth year'] > 1899, 'Birth year'].median()
dataset.loc[dataset['Birth year'] <= 1899, 'Birth year'] = birth_year_median

#create ['Start year'] column to extract the year of the original start date
dataset['Start year'] = dataset['Employment Details Original Start Date'].dt.year

#create ['Age'] column to calculate the age of the employee at the time of the rating
dataset['Age'] = dataset['Year_x'] - dataset['Birth year']

#catch outliers in ['Age'] column and impute with mean
age_mean = dataset.loc[(dataset['Age'] >= 16) & (dataset['Age'] < 124), 'Age'].mean()
dataset.loc[dataset['Age'] <= 16, 'Age'] = age_mean

#create ['Tenure'] column to calculate the tenure of the employee at the time of the rating (actives)
dataset['Tenure'] = dataset['Year_x'] - dataset['Start year']

#create ['Tenure'] column to calculate the tenure of the employee at the time of the rating (terminated)
dataset['Tenure'] = np.where(dataset['Employment Details Termination Date'].notnull(),
                        (dataset['Employment Details Termination Date'] - dataset['Employment Details Original Start Date']).dt.days / 365,
                        dataset['Tenure'])

#prepare for creation of ['Age Group'] column
bins = [0, 20, 30, 40, 50, 60, np.inf]
labels = ['<20', '20-30', '30-40', '40-50', '50-60', '60+']

#create ['Age Group'] column to categorize the age of the employees
dataset['Age Group'] = pd.cut(dataset['Age'], bins = bins, labels = labels, right = False)

#catch outliers in ['Tenure'] column, impute wrong values with the mean
tenure_mean = dataset.loc[dataset['Tenure'] >= 0, 'Tenure'].mean()
dataset.loc[dataset['Tenure'] <= 0, 'Tenure'] = tenure_mean

#create ['Age at start'] column to calculate the age of the employee at the time of the original start date
dataset['Age at start'] = dataset['Start year'] - dataset['Birth year']

#catch outliers in ['Age at start'] and impute with mean
startage_mean = dataset.loc[dataset['Age at start'] >= 16, 'Age at start'].mean()
dataset.loc[dataset['Age at start'] <= 16, 'Age at start'] = startage_mean

#create mode date to fill ['Position since'] missing values and impute with mode
dataset['Position since'].fillna(dataset['Position since'].mode()[0], inplace=True)

#define []'rating'] mapping, map ratings to numerical values
rating_mapping = {
    'Unrated': 0, 'Unsatisfactory': 1, 'Developing': 2, 'Performing': 3, 'Exceeding': 4, 'Outstanding': 5
}

dataset['rating_num'] = dataset['rating'].map(rating_mapping)

#lets try imputing ['FTE'] with 1.0000 instead (mode didnt work, mean not accurate enough, 1.0000 close enough)
dataset['FTE'].fillna(1.0000, inplace = True)

#check and impute negative ['Direct Reports'], we impute negative ones with 0 till we know more
dataset.loc[dataset['Direct Reports'] < 0, 'Direct Reports'] = 0

#rename ['Year_y'] to ['Year']
dataset.rename(columns={'Year_y': 'Year'}, inplace=True)

#describe to check the distribution of numerical features
dataset.info()

In [ ]:
dataset['Country (LE)'].unique()

In [ ]:
no_gender = dataset[dataset['Gender'].isnull()]
no_gender

In [ ]:
#check for duplicates and remove such
dataset.duplicated().sum()

dataset = dataset.drop_duplicates()

In [ ]:
#####################################################
'''SENSITIVE DATA & REMOVAL OF UNNECESSARY COLUMNS'''
#####################################################

dataset = dataset.drop(['User/Employee ID', 'Username_x', 'Date Of Birth', 'Username_y', '^UserId',
                        'In current position since', 'Employment Details Calculation Start Date',
                        '2. Citizenship', 'Employment Details Termination Date', 'Employment Details Original Start Date',
                        'Employment Details Hire Date', 'Position', 'Position Level', 'New Job since', 'Global Function',
                        'Job Title', 'start-date', 'end-date', 'Year_x', '^AssignmentId', 'composite_key',
                        'Employment Details Calculation Start Year'], axis='columns')

dataset.info()

In [ ]:
#final check for outliers in the datase
#store outliers in a dictionary
outliers = {}

#check for numericallity and attach outliers to the dictionary
for column in dataset.columns:
    if dataset[column].dtype in ['int64', 'float64']:
        outlier_df = calculate_modified_z_score(dataset, column)
        outliers[column] = len(outlier_df)

outliers

In [ ]:
'''save the dataset twice, once for gender imputation and once for the final dataset'''

dataset.to_csv(gender_file_path, index = False)
dataset.to_csv(dataset_file_path, index = False)